# Binomial Tree - Cox, Ross, Rubinstein model

## Binomial Tree method to calculate the price of an European or American Option given u, d and number of periods

### (1+r) per period compounding

n : number of periods <br/>
r_f: risk free rate <br/>
S_0: initial stock price <br/>
K: strike price <br/>
u: up-factor <br/>
d: down-factor <br/>
PutCall: Type P for put, C for call <br/>
Type: Type = E for European, = A for American (default is European) <br/>

If volatility is to be used then the calculation is: <br/>
$u = \exp(\sigma \sqrt{\frac{T}{n}})$ <br/>
$d = 1/u$


In [73]:
import numpy as np

The stock_lattice function calculates the lattice of possible stock price movements in the binomial tree. This is created as a separate function since it would be used later in many pricing codes which involves a binomial tree.

In [76]:
def stock_lattice(S_0,n,u,d):
    
    #S_0 = initial stock price
    #n: number of periods
    #u,d = up/down movement in each step
    
    stock_price = np.zeros((n+1,n+1))
    stock_price[0,0] = S_0
    
    for i in range(1,n+1):
        stock_price[i,0] = stock_price[i-1,0]*d
        for j in range(1,i+1):
            stock_price[i,j] = stock_price[i-1,j-1]*u
        
        
    return stock_price

In [77]:
def binomial_tree(n, r_f, S_0, K, u, d, PutCall, Type):
    
    #n:number of periods
    #r_f: risk free rate
    #S_0:initial stock price
    #K:strike
    #u,d : up and down factor
    #PutCall: enter 'Put' for Put Option, 'Call' for Call Option
    #Type: enter 'E' for European and 'A' for American
    
    q = ((1.0 + r_f) - d)/(u-d)    #the risk-neutral probability
    
    stock_price = stock_lattice(S_0, n, u, d)
    
    option_price = np.zeros((n+1,n+1))
    
    #calculating the price at the terminal node
    
    for i in range(n+1):
        if PutCall=="Call": # Call
            option_price[n,i] = max(0, stock_price[n,i]-K)
        elif PutCall=="Put": #Put
            option_price[n,i] = max(0, K-stock_price[n,i])
            
    
    if Type == 'E':  #for a European Option we do not check for early exercise
        
         for i in range(n-1,-1,-1):
            for j in range(i+1):
                option_price[i,j] = (1/(1+r_f))*(q*option_price[i+1,j+1]+(1-q)*option_price[i+1,j])
                
    elif Type == 'A': #for an American Option the value is the maximum of binomial price and early exercise payoff
        
         for i in range(n-1,-1,-1):
            for j in range(i+1):
                if PutCall=="Put":
                    option_price[i,j] = max(0, K - stock_price[i,j], 
                                           (1/(1+r_f))*(q*option_price[i+1,j+1]+(1-q)*option_price[i+1,j]))
                elif PutCall=="Call":
                    option_price[i,j] = max(0, stock_price[i,j] - K, 
                                           (1/(1+r_f))*(q*option_price[i+1,j+1]+(1-q)*option_price[i+1,j])) 
    
    
    
    return option_price[0,0]
    
    
    

In [80]:
binomial_tree(3, 0.01, 100, 100, 1.07, 1.0/1.07, 'Put', 'A')

3.823930814466083